
# Módulo 2: Scraping con Selenium
## LATAM Airlines

<a href="https://www.latam.com/es_ar/\"> 
<img src="https://i.pinimg.com/originals/dd/52/74/dd5274702d1382d696caeb6e0f6980c5.png" width="420"></img>
</a><br>

Vamos a scrapear el sitio de Latam para averiguar datos de vuelos en funcion el origen y destino, fecha y cabina. La información que esperamos obtener de cada vuelo es:

* Precio(s) disponibles
* Horas de salida y llegada (duración)
* Información de las escalas
¡Empecemos! Utilicemos lo aprendido hasta ahora para lograr el objetivo propuesto



In [0]:
import requests
from bs4 import BeautifulSoup

In [0]:
url = 'https://www.latam.com/es_co/apps/personas/booking?fecha1_dia=01&fecha1_anomes=2020-07&from_city1=BOG&to_city1=CLO&ida_vuelta=ida&cabina=W&nadults=1&nchildren=0&ninfants=0&app=deal-finder#/'

In [20]:
request = requests.get(url)
request.status_code

200

In [0]:
soup = BeautifulSoup(request.text, 'lxml')

In [22]:
print(soup.prettify())

<!DOCTYPE html>
<html lang="es">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="IE=edge,chrome=1" http-equiv="X-UA-Compatible"/>
  <title>
   Selecciona tus vuelos | LATAM Airlines
  </title>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="https://s.latamstatic.com/static/latam/images/favicon/LATAM/favicon.ico" rel="shortcut icon" type="image/x-icon"/>
  <meta content="personas_bs3_latam_mu" name="layout"/>
  <meta content="LATAM.com" name="description"/>
  <meta content="LATAM.com" name="keywords"/>
  <script src="https://www.pontosmultiplus.com.br/cestaautenticacao/app/multiplus-sdk.js" type="text/javascript">
  </script>
  <meta content="DD755E358AE0730E63D5C97A163CB111" name="sessionId" scheme="tracker"/>
  <meta content="latam" name="matchedScenario" scheme="tracker"/>
  <meta content="personas_bs3_latam_mu" name="template" scheme="tracker"/>
  <meta content="null" name="userDataCookieRawData"

Vemos que la respuesta de la página no contiene la información que buscamos, ya que la misma aparece recién después de ejecutar el código JavaSCript que está en la respuesta.



## Selenium

Selenium es una herramienta que nos permitirá controlar un navegador y podremos utilizar las funcionalidades del motor de JavaScript para cargar el contenido que no viene en el HTML de la página. Para esto necesitamos el módulo webdriver.

In [23]:
!pip install selenium
!apt-get update # to update ubuntu to correctly run apt install

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease
Hit:10 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 Packages [1,226 kB]
Get:15 http://arc

In [24]:
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver /usr/bin

Reading package lists... Done
Building dependency tree       
Reading state information... Done
chromium-chromedriver is already the newest version (81.0.4044.138-0ubuntu0.18.04.1).
0 upgraded, 0 newly installed, 0 to remove and 52 not upgraded.
cp: '/usr/lib/chromium-browser/chromedriver' and '/usr/bin/chromedriver' are the same file


In [0]:
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

In [0]:
from selenium import webdriver

**Paso 1:** Instanciamos un **driver** del navegador

In [31]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


**Paso 2:** hacer que el navegador cague la página web

In [0]:
driver.get(url)

**Paso 3:** Extraer la información de la página web

In [0]:
# El xpath (ruta xml) nos servirá para seleccionar los elementos en el navegador
# El xpath nos permite identificar donde se encuentra cada elemento
# Con el xpath podemos obtener toda la info que necesitamos 

# // Seleccionar elementos directos e indirectos del nodo
flights = driver.find_elements_by_xpath('//li[@class="flight"]')

In [36]:
flight = flights[0]
flight

<selenium.webdriver.remote.webelement.WebElement (session="8035e6d205b5ff803adb1be309ace769", element="a87e0700-2c56-469f-ba9b-a4dd7b22915a")>

In [0]:
def get_flight_departure_time(flight):
  #  El . es Para buscar del vuelo hacia abajo
  # Hora de salida
  return flight.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')

In [0]:
def get_flight_arrival_time(flight):
  # Hora de llegada
  return flight.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')

In [0]:
def get_flight_duration(flight):
  # Duración del vuelo
  return flight.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')

In [52]:
scales_button = flight.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
scales_button

<selenium.webdriver.remote.webelement.WebElement (session="8035e6d205b5ff803adb1be309ace769", element="8668cb0f-c62f-4a09-bcb5-03d1997ebb33")>

In [0]:
from selenium.webdriver.common.action_chains import ActionChains

#scales_button.click()
ActionChains(driver).move_to_element(scales_button).click().perform()

In [100]:
segments = flight.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
segments

[<selenium.webdriver.remote.webelement.WebElement (session="8035e6d205b5ff803adb1be309ace769", element="4390a901-6fc9-49f7-92a4-f54f520eaebd")>]

In [101]:
length_scales = len(segments)
print(f'El número de escalas es {length_scales}')

El número de escalas es 1


In [0]:
segment = segments[0]

In [0]:
def get_origin(segment):
  # Origen 
  return segment.find_element_by_xpath('.//span[@class="sc-bsbRJL bMMExG"]//abbr[@class="sc-hrWEMg hlCkST"]').text

In [0]:
def get_departure_time(segment):
  # Hora de salida
  return segment.find_element_by_xpath('.//span[@class="sc-bsbRJL bMMExG"]//time').text

In [0]:
def get_departure_airport(segment):
  # Aeropuerto de salida
  return segment.find_element_by_xpath('.//span[@class="sc-bsbRJL bMMExG"]/div[@class="sc-iujRgT jEtESl"]/span[@class="sc-eTuwsz eumCTU"]/span[@class="sc-hXRMBi gVvErD"]').text

In [0]:
def get_segment_duration(segment):
  # Duración
  return segment.find_element_by_xpath('.//span[@class="sc-cmthru ipcOEH"]//time').get_attribute('datetime')

In [0]:
def get_destination(segment):
  # Destino 
  return segment.find_elements_by_xpath('.//span[@class="sc-bsbRJL bMMExG"]//abbr[@class="sc-hrWEMg hlCkST"]')[-1].text

In [0]:
def get_airport_destination(Segment):
  # Aeropuerto Destino
  return segment.find_elements_by_xpath('.//span[@class="sc-bsbRJL bMMExG"]//span[@class="sc-eTuwsz eumCTU"]//span[@class="sc-hXRMBi gVvErD"]')[-1].text

In [0]:
def get_arrival_time(segment):
  # Hora llegada 
  return segment.find_elements_by_xpath('.//span[@class="sc-bsbRJL bMMExG"]//time')[-1].text

In [0]:
def get_flight_number(segment):
  # Número de vuelo
  return segment.find_element_by_xpath('.//div[@class="airline-flight-details"]//b').text

In [0]:
def get_airbus(segment):
  # Airbus
  return segment.find_element_by_xpath('.//div[@class="airline-flight-details"]//span[@class="sc-gzOgki uTyOl"]').text

In [0]:
close_button = segment.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]')
ActionChains(driver).move_to_element(close_button).click().perform()

In [0]:
flight.click()

In [93]:
tariffs = flight.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
tariffs

[<selenium.webdriver.remote.webelement.WebElement (session="8035e6d205b5ff803adb1be309ace769", element="35330c27-b77c-432f-add2-37b8e96ce901")>,
 <selenium.webdriver.remote.webelement.WebElement (session="8035e6d205b5ff803adb1be309ace769", element="7369eed8-19b6-478c-ae5a-47e0136388d6")>]

In [97]:
prices = []

for tariff in tariffs:
  name = tariff.find_element_by_xpath('.//label').get_attribute('for')
  currency = tariff.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
  price = tariff.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text
  dict_tariff = {name:{'currency':currency, 'price':price}}
  prices.append(dict_tariff)
  print(dict_tariff)

{'PLUS': {'currency': '$', 'price': '196.480'}}
{'TOP': {'currency': '$', 'price': '274.700'}}


In [0]:
def get_prices(flight):
  tariffs = flight.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')

  prices = []

  for tariff in tariffs:
    name = tariff.find_element_by_xpath('.//label').get_attribute('for')
    currency = tariff.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
    price = tariff.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text
    dict_tariff = {name:{'currency':currency, 'price':price}}
    prices.append(dict_tariff)
    #print(dict_tariff)
  return prices

In [0]:
def get_scales(flight):
  segments = flight.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
  scales_info = []
  for segment in segments:
    origin = get_origin(segment)
    departure_time = get_departure_time(segment)
    destination = get_destination(segment)
    arrival_time = get_arrival_time(segment)
    flight_duration = get_segment_duration(segment)
    flight_number = get_flight_number(segment)
    airbus = get_airbus(segment)

    data_dict = {
        'origin': origin,
        'departure_time': departure_time,
        'destination':destination,
        'arrival_time': arrival_time,
        'flight_duration': flight_duration,
        'flight_number': flight_number,
        'airplane_model': airbus
    }

    scales_info.append(data_dict)
  return scales_info

In [0]:
def get_times(flight):
  departure_time = get_flight_departure_time(flight)
  arrival_time = get_flight_arrival_time(flight)
  duration =  get_flight_duration(flight)

  times = {
      'departure_time': departure_time,
      'arrival_time': arrival_time,
      'duration': duration
  }
  return times


In [0]:
def get_info(driver):
  flights = driver.find_elements_by_xpath('//li[@class="flight"]')

  print(f'Se encontrarón {len(flights)}')
  print(f'Iniciando scrapping...')
  info = []
  for flight in flights:
    # Get the times each flight
    times = get_times(flight)

    # Click Scales Button
    scales_button = flight.find_element_by_xpath('.//div[@class="flight-summary-stops-description"]/button')
    ActionChains(driver).move_to_element(scales_button).click().perform()

    scales = get_scales(flight)

    # Close scales 
    close_button = driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]')
    ActionChains(driver).move_to_element(close_button).click().perform()

    # Click on flight for tariffs information
    flight.click()
    prices = get_prices(flight)
    flight.click()

    info.append({'prices':prices, 'times':times, 'scales': scales})
  return info

In [163]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--incognito')
chrome_options.add_argument('--headless')
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

driver.get(url)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:6: DeprecationWarning: use options instead of chrome_options
  


In [164]:
get_info(driver)

Se encontrarón 1
Iniciando scrapping...


[{'prices': [{'PLUS': {'currency': '$', 'price': '196.480'}},
   {'TOP': {'currency': '$', 'price': '274.700'}}],
  'scales': [{'airplane_model': 'Airbus 319',
    'arrival_time': '08:09',
    'departure_time': '07:03',
    'destination': 'CLO',
    'flight_duration': '1:6',
    'flight_number': 'LA4057',
    'origin': 'BOG'}],
  'times': {'arrival_time': '08:09',
   'departure_time': '07:03',
   'duration': 'PT1H6M'}}]

**Paso 4:** Cerrar el navegador

In [0]:
driver.close()

In [0]:
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import  By
from selenium.common.exceptions import TimeoutException

In [0]:
def run_scrapper():
  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--incognito')
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  driver = webdriver.Chrome('chromedriver',chrome_options=chrome_options)

  driver.get(url)
  # Cuando introducimos una demora es mejor hacer demoras dinamicas. Es decir que esperan a que aparezca
  # Algún elemento para continuar con el resto del código
  delay = 10
  flights_info = None
  try:
    # Esperar que cargue un vuelo
    # Esperamos hasta la presencia de algún elemento - WebDriverWait(driver, delay).until(EC.presence_of_all_elements_located)
    # Esperamos a que aparezca un vuelo con EC.presence_of_all_elements_located((By.XPATH, '//li[@class="flight"]')
    flight = WebDriverWait(driver, delay).until(EC.presence_of_all_elements_located((By.XPATH, '//li[@class="flight"]')))
    flights_info = get_info(driver)
  except TimeoutException as e:
    print('La página ha ternado demasiado en cargar')
    print(e)
    flights_info = []

  driver.close()
  return flights_info



In [168]:
run_scrapper()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: use options instead of chrome_options
  import sys


Se encontrarón 1
Iniciando scrapping...


[{'prices': [{'PLUS': {'currency': '$', 'price': '196.480'}},
   {'TOP': {'currency': '$', 'price': '274.700'}}],
  'scales': [{'airplane_model': 'Airbus 319',
    'arrival_time': '08:09',
    'departure_time': '07:03',
    'destination': 'CLO',
    'flight_duration': '1:6',
    'flight_number': 'LA4057',
    'origin': 'BOG'}],
  'times': {'arrival_time': '08:09',
   'departure_time': '07:03',
   'duration': 'PT1H6M'}}]